In [ ]:
import datetime as dt
import pandas as pd
import matplotlib
import numpy as np
import statsmodels
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import ccxt
import pandas as pd
import requests
import json
import time
from sklearn.model_selection import cross_val_score
from sklearn import datasets
from sklearn import svm
from sklearn.model_selection import TimeSeriesSplit 


In [ ]:
from datetime import datetime, timedelta

# Base Ethereum symbol
base_symbol = 'ETH'

# List of expiration dates
expiration_dates = [
    '2024-01-31',
    '2024-02-01',
    '2024-02-02',
    '2024-02-09',
    '2024-02-16',
    '2024-02-23',
    '2024-03-29',
    '2024-04-26',
    '2024-06-28',
    '2024-09-27',
    '2024-12-27'
]

# Function to generate option symbols for a given expiration date and strike prices
def generate_eth_option_symbols(expiration_date, strike_prices):
    # Convert expiration date string to datetime object
    exp_date = datetime.strptime(expiration_date, '%Y-%m-%d')

    # List to store generated symbols
    symbols = []

    for strike_price in strike_prices:
        # Call option symbol
        call_option_symbol = f'{base_symbol}-{exp_date.strftime("%y%m%d")}-{strike_price}-C'
        symbols.append(call_option_symbol)

        # Put option symbol
        put_option_symbol = f'{base_symbol}-{exp_date.strftime("%y%m%d")}-{strike_price}-P'
        symbols.append(put_option_symbol)

    return symbols

# Provided strike prices
strike_prices = [
    2100, 2150, 2175, 2200, 2225, 2250,
    2275, 2300, 2325, 2350, 2374.1, 2375,
    2400, 2425, 2450, 2475
]

# Generate option symbols for each expiration date
for expiration_date in expiration_dates:
    symbols = generate_eth_option_symbols(expiration_date, strike_prices)
    print(f"Symbols for {expiration_date}: {symbols}\n")
symbols

In [ ]:
import requests
url = "https://api.tokenterminal.com/v2/projects"
headers = {
    "accept": "application/json",
    "authorization": TOKENTERMINAL_API_KEY
}
response = requests.get(url, headers=headers)
print(response.text)
list_of_dicts= eval(response.text[8:-1])
List_all_proj_id= pd.DataFrame.from_dict(list_of_dicts)
List_all_proj_id[List_all_proj_id['name'].str.startswith('Eth')]

In [ ]:
Eth_merge= dt.date(2022, 9, 15)

In [ ]:
url = 'https://www.deribit.com/api/v2/public/get_book_summary_by_currency'
params = {
    'currency': 'ETH',
    'kind' : 'option'
}

response = requests.get(url, params=params)
data = response.json()

In [ ]:
Options_Book= pd.DataFrame.from_dict(data['result']).drop(['quote_currency', 'base_currency', 'underlying_index'], axis = 1).dropna()
Options_Book.set_index(Options_Book['instrument_name'], inplace=True)
Options_Book.drop(['instrument_name', 'estimated_delivery_price', 'interest_rate', 'ask_price', 'bid_price', 'mid_price','last', 'low', 'creation_timestamp', 'high', 'volume_usd'], axis=1, inplace=True)
Options_Book.apply(pd.to_numeric).dropna(inplace=True)
most_liquid_ops = Options_Book.sort_values(by = "volume", ascending = False).head(15)
most_liquid_ops

In [ ]:
current_timestamp_ms = int(time.time()) * 1000
options_price_history = {}
options_price_history.keys() == [most_liquid_ops]
price_his_df = pd.DataFrame(columns = most_liquid_ops)
for op in most_liquid_ops:
    params = {
    'instrument_name': op,
    'start_timestamp': int(current_timestamp_ms - (len(pd.date_range(end = dt.date.today(), start = dt.date(2021, 5, 12))) * 24 * 60 * 60 * 1000)),
    'end_timestamp': current_timestamp_ms,
    'count': 1000,
    'resolution' : 'D'
         }
    op_mark_price_hist = requests.get("https://www.deribit.com/api/v2/public/get_mark_price_history", params).json()
    try:
        options_price_history.update({op : pd.DataFrame(op_mark_price_hist['result']).set_index(pd.to_datetime(pd.DataFrame(op_mark_price_hist['result'])[0], unit = 'ms')).drop(0, axis=1)})
    # price_his_df.concat(options_price_history[op], orient = 'index')
    except:
        continue
    

In [ ]:
current_timestamp_ms = int(time.time()) * 1000
Deribit_data = {}
URLs = {
        'bvolU' : 'https://www.deribit.com/api/v2/public/get_volatility_index_data',
        'histvol' : 'https://www.deribit.com/api/v2/public/get_historical_volatility?currency=ETH',
        'op_bk' : 'https://www.deribit.com/api/v2/public/get_book_summary_by_currency'
}
params = {
    'currency': 'ETH',
    'start_timestamp': current_timestamp_ms -  (1000 * 24 * 60 * 60 * 10000),  # Since Eth merge converted to timestamp ms
    'end_timestamp': current_timestamp_ms,
    'resolution': 'D'
    # (len(pd.date_range(end = dt.date.today(), start = Eth_merge))
}
for (name, url) in URLs.items():
    Deribit_data.update({name : requests.get(url, params).json()})
BVOL = pd.DataFrame(Deribit_data['bvolU']['result']['data'])
BVOL.set_index(pd.to_datetime(BVOL[0], unit = 'ms'), inplace = True)
BVOL.drop(0, axis=1, inplace = True)
BVOL= pd.DataFrame(BVOL.mean(axis=1))
BVOL.columns = ['BVOL']
HVOL = pd.DataFrame(Deribit_data['histvol']['result'])
HVOL.set_index(pd.to_datetime(HVOL[0], unit = 'ms'), inplace = True)
HVOL.drop(0, axis =1, inplace =True)
HVOL.columns = ['HVOL']
Deri_Vola= BVOL.merge(HVOL, left_index = True, right_index = True)
OPBK= pd.DataFrame(Deribit_data['op_bk']['result'])
Deri_Vola.index = pd.to_datetime(Deri_Vola.index.date)

In [ ]:
url = "https://api.tokenterminal.com/v2/projects/ethereum/metrics"

headers = {
    "accept": "application/json",
    "authorization": TOKENTERMINAL_API_KEY
}

response = requests.get(url, headers=headers)
raw_data = response.text.replace("null", "None")
raw_data = raw_data[response.text.find("['")+9:response.text.find("']")]
list_of_df = eval(raw_data)
Full_on_chain = pd.DataFrame.from_dict(list_of_df).dropna()
Full_on_chain.set_index(pd.to_datetime(Full_on_chain['timestamp']), inplace= True)
Full_on_chain.drop("timestamp", axis =1, inplace= True)
# Full_on_chain= Full_on_chain[Full_on_chain.index.date >= Eth_merge]
Full_on_chain['Log_returns']= np.log(Full_on_chain["price"])-np.log(Full_on_chain['price']).shift(-1)
Full_on_chain['Exp_weight_std'] = Full_on_chain['Log_returns'].sort_index(ascending = True).ewm(alpha = 0.025).std()*Full_on_chain['price']
Full_on_chain.index = pd.to_datetime(Full_on_chain.index.date)
# Full_on_chain= Full_on_chain[Full_on_chain.index > dt.datetime(2023,1,1)]
Full_on_chain = Full_on_chain.merge(BVOL, left_index = True, right_index =True)
FON_st= Full_on_chain.merge(HVOL, left_index=True, right_index=True)
FON= Full_on_chain.drop(["project_name", "project_id",'market_cap_fully_diluted'], axis = 1).apply(pd.to_numeric)

In [ ]:
Forecast_horizon= 14
tss = TimeSeriesSplit(test_size= Forecast_horizon, n_splits = int(len(X)/Forecast_horizon -1))
scores = []
preds= []
df_to_use= FON.sort_index()
X = df_to_use
# .drop(['ps_fully_diluted', 'pf_fully_diluted', 'token_turnover_fully_diluted', 'fees_supply_side', 'transaction_fee_average', 'market_cap_circulating'], axis = 1) 
Y = df_to_use["price"].shift(-Forecast_horizon).dropna()

for (train , test) in tss.split(Y):
    X_train = X.iloc[train]
    Y_train = Y.iloc[train]
    X_test = X.iloc[test]
    Y_test = Y.iloc[test]
    xgbmodel = xgb.XGBRegressor(n_estimators=1000, early_stopping_rounds=100).fit(X_train, Y_train, eval_set = [(X_train, Y_train), (X_test, Y_test)], verbose=False)
    Y_pred= xgbmodel.predict(X_test)
    preds.append(y_pred)
    score= np.sqrt(mean_squared_error(Y_test, Y_pred)) / Y_pred
    scores.append(score)
print(np.mean(scores), np.std(scores))
pd.concat([pd.DataFrame(Y_pred, index= Y_test.index, columns= ['prediction']), Y_test.rename("realized")], axis=1).plot()
Y_train = Y.iloc[: - Forecast_horizon]
X_train = X.loc[Y_train.index]
Y_test = Y.iloc[-Forecast_horizon: ]
X_test = X.loc[Y_test.index]
xgbmodel.fit(X_train, Y_train, eval_set = [(X_train, Y_train), (X_test, Y_test)], verbose=False)
Y_pred= xgbmodel.predict(X_test)
score= np.sqrt(mean_squared_error(Y_test, Y_pred)) / Y_pred
score.mean(), score.std()
# Y.dropna(inplace=True)#predict with todays data tmr's change in BVOL
# X.dropna(inplace=True)
# X_weekly = X.rolling(7).mean()
# Y_weekly = Y.rolling(7).mean()
# Y_pred_1_week = Y_weekly.shift(-7)
# Y_pred_1_week.dropna(inplace=True)
# Y_pred_1_week.index = pd.to_datetime(Y_pred_1_week.index)

In [ ]:
Y.plot(legend= True)
Y_test.plot()
X['price'].plot()
X_test['price'].plot()
pd.concat([pd.DataFrame(Y_pred, index= Y_test.index, columns= ['prediction']), Y_test.rename("realized")], axis=1).plot()

In [ ]:
Correlation_Matrix_lagged_X= pd.concat([X,Y], axis =1).corr()
Cor_mat_diff = pd.concat([X,Y], axis =1).diff().corr()
# Corr_m= pd.DataFrame(Correlation_Matrix_lagged_X['BVOL'].apply(pd.to_numeric).apply(np.abs))
Corr_m_d= pd.DataFrame(Cor_mat_diff['BVOL'].apply(pd.to_numeric).apply(np.abs))
Feature_selec= pd.DataFrame(xgbmodel.feature_importances_, index = xgbmodel.feature_names_in_).sort_values(by = 0, ascending = False).merge(Correlation_Matrix_lagged_X, left_index = True, right_index = True, how ='outer')
Feature_selec.rename({0: "feature_importance"}, inplace= True, axis =1)
Feature_selec.sort_values("feature_importance", ascending=False)
Feature_selec_standarized = Feature_selec.mul(1/Feature_selec.mean()).sort_index()
Feature_selec_standarized['corr_with_price'] =  pd.DataFrame(Cor_mat_diff['price'].apply(pd.to_numeric).apply(np.abs))
Feature_selec_standarized['mean_corr_with_others'] = -(Correlation_Matrix_lagged_X.mean().sort_index().values)
Feature_selec_standarized = Feature_selec_standarized.mul(1/Feature_selec.mean()).sort_index()
Features= Feature_selec_standarized.sum(axis=1).sort_values(ascending=False).head(17).keys().drop_duplicates()
X = X[Features]
Y_train = Y.iloc[: - Forecast_horizon]
X_train = X.loc[Y_train.index]
Y_test = Y.iloc[-Forecast_horizon: ]
X_test = X.loc[Y_test.index]
xgbmodel.fit(X_train, Y_train, eval_set = [(X_train, Y_train), (X_test, Y_test)], verbose=False)
Y_pred= xgbmodel.predict(X_test)
score= np.sqrt(mean_squared_error(Y_test, Y_pred)) / Y_pred
print(score.mean(), score.std())
# for values in Feature_selec.values:
#     fea_scores.append(np.mean(values))
# pd.concat([Feature_selec, pd.DataFrame(fea_scores, index= Feature_selec.index)], axis = 1)
# Feature_selec.columns